# Notebook to experiment with ways to quantify pec fin morphology

### Import packages

In [1]:
from ome_zarr.io import parse_url
from ome_zarr.reader import Reader
import napari
import numpy as np
from napari_animation import Animation
import pandas as pd

### Now, load an image dataset along with nucleus masks inferred using cellpose.

In [2]:
import warnings
warnings.filterwarnings("ignore")

# set parameters
filename = "2022_12_15 HCR Hand2 Tbx5a Fgf10a_1"
readPath = "/Users/nick/Dropbox (Cole Trapnell's Lab)/Nick/pecFin/HCR_Data/built_zarr_files/" + filename + '_nucleus_props.csv'

image_df = pd.read_csv(readPath)
image_df.head()


,Unnamed: 0,Z,Y,X,Area,Axis_1,Axis_2,Axis_3,axis_dirs
0,0,2.639764,1.546455,84.725788,2537,10.984421,3.961844,1.871490,[[ 0.61748594 0.71242812 -0.33340261]\n [-0.3...
1,1,1.705905,5.637094,86.416871,5084,13.043954,5.298203,2.406970,[[-0.96889441 -0.21164702 0.1282543 ]\n [-0.1...
2,2,5.189623,9.904291,96.172451,20784,23.681668,6.666189,4.589988,[[ 0.06119428 0.99808033 0.00953524]\n [ 0.9...
3,3,2.486621,7.913299,93.042876,5522,13.582985,4.772553,2.939320,[[-0.99538174 -0.06800537 0.06775296]\n [-0.0...
4,4,1.038200,10.568959,83.740526,2672,12.182761,4.115927,1.688992,[[-0.99384776 -0.10485687 -0.03566044]\n [ 0.0...


### convert to point cloud and import pacakges

In [5]:
import numpy as np
import open3d as o3d
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans, DBSCAN, OPTICS
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.asarray(image_df.iloc[:, 0:3]))
# Get points and transform it to a numpy array:
points = np.asarray(pcd.points).copy()

### Experiment with basic k-means classifier

In [16]:
# Normalisation:
scaled_points = StandardScaler().fit_transform(points)
# Clustering:
model = KMeans(n_clusters=4)
model.fit(scaled_points)

# Get labels:
labels = model.labels_
# Get the number of colors:
n_clusters = len(set(labels))

# Mapping the labels classes to a color map:
colors = plt.get_cmap("tab20")(labels / (n_clusters if n_clusters > 0 else 1))
# Attribute to noise the black color:
colors[labels < 0] = 0
                                  

ValueError: too many values to unpack (expected 2)

In [24]:
labels_u, i_u = np.unique(labels, return_index=True)
color_names = ['blueviolet', 'lightgreen', 'salmon']
fig = px.scatter_3d(image_df.iloc[labels==labels_u[0]], x="X", y="Y", z="Z",opacity=0.5)
fig.update_traces(marker=dict(color=colors[i_u[0]]))
for i in range(1, len(labels_u)):
    fig.add_trace(
                go.Scatter3d(
                    mode='markers',
                    x=image_df["X"].iloc[labels==labels_u[i]],
                    y=image_df["Y"].iloc[labels==labels_u[i]],
                    z=image_df["Z"].iloc[labels==labels_u[i]],
                    opacity=5,
                    marker=dict(
                        color=colors[i_u[i]],
                        size=7
                    ),
                    showlegend=False)
                )
fig.show()  

ValueError: 
    Invalid value of type 'builtins.int' received for the 'opacity' property of scatter3d
        Received value: 5

    The 'opacity' property is a number and may be specified as:
      - An int or float in the interval [0, 1]

In [ ]:
labels_u, i_u = np.unique(labels, return_index=True)
color_names = ['blueviolet', 'lightgreen', 'salmon','honeydew']
fig = px.scatter_3d(image_df.iloc[labels==labels_u[0]], x="X", y="Y", z="Z",opacity=0.5, color=color_names[0])
for i in range(1, len(labels_u)):
    fig.add_trace(
                go.Scatter3d(image_df.iloc[labels==labels_u[i]],
                    mode='markers',
                    x="X", y="Y", z="Z",
                    marker=dict(
                        color=color_names[i],
                        size=7
                    ),
                    showlegend=False)
                )
fig.show()  